In [37]:
# Import packages
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from tensorflow.keras import layers
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [5]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [7]:
# Import the pkl file.
with open('ai_image_classifier_small_img.pkl', 'rb') as file:
    recalled_imgs = pickle.load(file)

type(recalled_imgs)

dict

In [8]:
# Set train, test variables.
X_train = recalled_imgs['X_train']
X_test = recalled_imgs['X_test']
y_train = recalled_imgs['y_train']
y_test = recalled_imgs['y_test']

In [9]:
# Convert values to numpy arrays
X_train_aug_np = np.array(X_train)
X_test_np = np.array(X_test)
y_train_aug_np = np.array(y_train) 
y_test_np = np.array(y_test)

# Load and preprocess your CMU Face Images dataset (Ensure each image is labeled as "with sunglasses" or "without sunglasses")
# The following code assumes that you have already loaded and preprocessed your dataset into 'X' and 'y' (features and labels).

# Split the training dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_aug_np, y_train_aug_np, test_size=0.2, random_state=42)

In [43]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, lr,  batch_size, epochs ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    optimizerD= {'Adam':Adam(learning_rate=lr), 'SGD':SGD(learning_rate=lr),
                 'RMSprop':RMSprop(learning_rate=lr), 'Adadelta':Adadelta(learning_rate=lr),
                 'Adagrad':Adagrad(learning_rate=lr), 'Adamax':Adamax(learning_rate=lr),
                 'Nadam':Nadam(learning_rate=lr), 'Ftrl':Ftrl(learning_rate=lr)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = optimizerD[optimizerL[round(optimizer)]]
        nn = Sequential()
        nn.add(layers.Conv2D(neurons, (3, 3), activation=activation, input_shape=(64, 64, 3)))
        nn.add(layers.MaxPooling2D((2, 2)))
        nn.add(layers.Flatten())
        nn.add(layers.Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]})
    score=np.nan_to_num(score)
    score=score.mean()
    return score

In [44]:
# Set paramaters
params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0, 7),
    'lr':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   |    lr     |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
| 1         | 0.1529    | 5.51      | 335.3     | 54.88     | 0.7716    | 36.58     | 1.044     |
| 2         | 0.09844   | 0.2023    | 536.2     | 39.09     | 0.3443    | 99.16     | 1.664     |
| 3         | 0.1239    | 0.7307    | 735.7     | 69.7      | 0.2815    | 51.96     | 0.8286    |
| 4         | 0.1178    | 0.6656    | 920.6     | 83.52     | 0.8422    | 83.37     | 6.937     |
| 5         | 0.1042    | 5.195     | 851.0     | 53.71     | 0.03717   | 50.87     | 0.7373    |
| 6         | 0.135     | 7.355     | 758.2     | 65.22     | 0.2815    | 99.86     | 0.9663    |
| 7         | 0.1463    | 5.539     | 588.0     | 52.4      | 0.7306    | 39.05     | 2.804     |
| 8         | 0.1223    | 2.871     | 957.8     | 93.5      | 0.8157    | 13.07     | 6.604     |
| 9         | 0.0984

In [45]:
params_nn_ = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_

{'activation': <LeakyReLU name=leaky_re_lu_4, built=True>,
 'batch_size': 332.36703829540227,
 'epochs': 58.6363629536105,
 'lr': 0.27010478764250984,
 'neurons': 25.501069518528467,
 'optimizer': 4.713791267287378}